 In fact, I’d argue that being able to successfully and safely test in production requires a significant amount of automation, a firm understanding of the best-practices as well as designing the systems from the ground up to lend themselves well toward this form of testing.

a/b testing is googd fore mountain not other montain mozilla what the baseline and how long can i check cant tell you if are missing something a/b test only things that dont take long analysis of logs before the ab testing we want to check its worth the change statistical and practical significant


A/B test's purpose is usually to see users' response (In a way, how much they like it) to a new UI, feature, etc. But you know that the new version works. So, you actually send randomly both versions of the application to all of them. It can be 50-50, 80-20, 90-10, anything. Sometimes the functionality is not even relevant. You might want to see which version attracts more clients and stuff like that.

Canary is more focused on how well works the new feature. Or if it actually works. It usually will be 90-10, 80-20, A >> B. Never 50-50, because if it goes wrong, you don't want half of your users to have a bad experience. So you are not positive if the new version is going to work as expected.

The most important difference (and this is what almost no one talks about) is that a canary testing has session affinity. So it doesn't send both versions to all users, but randomly sends some users to the new version, and keeps them on the same version.


Websites today are meticulously designed to maximize one or even several goals. Should the “Buy Now!” button be red or blue? What headline attracts the most clicks to that news article? Which version of an advertisement has the highest click-through rate? To determine the optimal answer to these questions, software developers employ A/B tests — a statistically sound technique to compare two different variants, version A and version B.

https://miro.medium.com/max/1400/1*LFnVVlLrgrvXQ5ScrHVKjg.jpeg

Which, in turn, is only feasible if deploying a service in the production environment doesn’t expose users to that service immediately.
In this post, I’ve decided to go with the terminology used in the blog post “Deploy != Release” 

When we say a version of a service is released, we mean that it is responsible for serving production traffic. In verb form, releasing is the process of moving production traffic to the new version.

A rollout is a logical unit of work that is composed of one or more individual tasks. Our goal is to fit the deployment process to the risk profile of a given service

For sensitive pieces of infrastructure, we may extend the rollout over several days, interleaving them across instances in different geographic regions.




Testing in Production during the Deploy Phase
	Integration Testing
		Traditionally, integration testing is performed by a CI server in an isolated “testing” environment over every git branch. A copy of the entire service topology (including databases, queues, caches, proxies and so forth) is spun up for the test suites of all the services to be run against each other.

		I believe this to be not terribly effective for several reasons. First of all, as with staging, it’s impossible for the test environment being spun up to be identical to the real production environment, even if the tests are run in the same Docker container that will be deployed to production

		There is sufficient research to corroborate that distributed systems often times exhibit emergent behavior, which cannot be predicted through analysis at any level simpler than that of the system as a whole.

		Integration testing is still pretty important in order to verify that a new version of a service does not:
		— break the contract it exposes to its upstreams or downstreams
		— affect the SLO of any of the upstreams or downstreams in an adverse way


	Shadowing (also known as Dark Traffic Testing or Mirroring)

		Systems behave differently depending on environment and traffic patterns. Since the behavior of utilization can change at any time, sampling real traffic is the only way to reliably capture the request path.

		Shadowing is the technique by which production traffic to any given service is captured and replayed against the newly deployed version of the service. This can happen either in real time where incoming production traffic is bifurcated and routed to both the released and deployed version, or it could happen asynchronously when a copy of the previously captured production traffic is replayed against the deployed service.

		The downside to shadowing the entire traffic against a deployed service in the production environment is that if such shadowing is done at the time of peak traffic, one might end up requiring 2X the capacity to perform such testing.


	Tap Compare 	Teeing

		Sending production requests to instances of a service in both production and staging environments and comparing the results for correctness and evaluating performance characteristics.

		The difference between tap compare and shadowing appears to be that in the former case, the response returned by the released version is compared with the response returned by the deployed version, whereas in the latter, the request is mirrored to the deployed version in a fire-and-forget manner.


	Load Testing



	Config Tests

		However, not testing new configuration before the release of code has been the cause of a significant number of outages.

		In order to test configuration changes holistically, it becomes important to draw the distinction between different forms of configuration. Fred Hebert, in a conversation with me, came up with the following quadrant:

		https://miro.medium.com/max/1400/1*M7YBU8Zym0EFA92sgaaS3A.png

Testing in Production — Release

It’s important here to stress that in this phase, a rollback happens only when there are hard failure modes

	Canarying

		Canarying refers to a partial release of a service to production. A subset of production now consists of the canaries which are then sent a small percentage of actual production traffic after they pass a basic health check. The canary population is monitored as it serves traffic, the metrics being monitored for are compared with the non-canary metrics (the baseline)


	Monitoring

		Monitoring is a must-have at every phase of the production rollout, but is especially important during the release phase. “Monitoring” is best suited to report the overall health of systems. Aiming to “monitor everything” can prove to be an anti-pattern. For monitoring to be effective, it becomes salient to be able to identify a small set of hard failure modes of a system or a core set of metrics. Examples of such failure modes are:


	Traffic Shaping

		Traffic shaping or traffic shifting really isn’t so much a standalone form of testing than something to assist with canarying and the gradual release of the new code. In essence, traffic shifting is achieved by updating the configuration of the load balancer to gradually route more traffic to the newly released version.



Testing in Production — Post-Release

Post-release testing takes the form of verification that happens once we’ve satisfactorily released code. 


	Feature Flagging or Dark Launch

		Once new code is deployed to production behind a feature flag, it can then be tested for correctness and performance as and when required in the production environment. Feature flagging is also one of the more accepted forms of testing in production, and as such is well-known and has been extensively written about. What’s perhaps less known is that feature flagging can be extended to testing database migrations or desktop software as well.

	A/B Testing
		A/B testing often falls under the experimental analysis and isn’t necessarily seen as a form of testing in production. As such it’s not only widely (sometimes controversially) used but also extensively researched and written about (including what makes good metrics for online experiments). What’s possibly less common is A/B testing being used for testing different hardware or VM configurations, but these are often referred to as “tuning” (for example, JVM tuning) as opposed to being seen as an A/B test (even if in principle, tuning can very much be seen as a form of A/B test to be undertaken with the same level of rigor when it comes to measurement).

		What’s possibly less common is A/B testing being used for testing different hardware or VM configurations, but these are often referred to as “tuning” (for example, JVM tuning) as opposed to being seen as an A/B test (even if in principle, tuning can very much be seen as a form of A/B test to be undertaken with the same level of rigor when it comes to measurement).


	Logs/Events, Metrics and Tracing
		https://copyconstruct.medium.com/monitoring-in-the-time-of-cloud-native-c87c7a5bfa3e

	Profiling
		Profiling applications in production is sometimes required to diagnose performance related problems.



	Chaos Engineering
		Chaos Engineering is the discipline of experimenting on a distributed system in order to build confidence in the system’s capability to withstand turbulent conditions in production.

logs = interactions between data and real worlds
data quality that compare two dataframes (or sizes )

- without metrics we do not have knowledge about the internal properties of the data itself making alerting based on real time anomaly detection on metrics imposible. if we only had metadata we would know the shape structure and timing of data but not necessarily whether we had bad data
- without metadata we do not know the structure or timing of data making use cases like schema change detection or satisfying SLA by tracking outages impossible. these use cases are critical to improve data reliability and decrease data downtime. if we only had metrics we would know whether the data is correct but not necessarily if it was refreshed in the apporipate amount of time
- without lineage we do not know how different pieces od data depend on each other maing it difficult to analyze upstream root cause and downstream impact of data quality issues. if we only had metrics and metadata we have holistic view of the health of data but incomplete picture of how issues are related
- without logs we do not know how external systems like elt and trasformation tools impact our data and how external users are impacfted by our data . with only data lineage we know hodata is related but not necessarilt how important those relationships are (does anyone use this data? ) or who is responsible for upstream changes ( whoc potentially caused the issue)
- 

1. until its feasible to store snaphost of data over time we rely on storing approximation of our data ala lossy compression
2. four categoris of charactaristics avout our data form the four pillars of ttdata objestability
3. the metrics and metadata pillar describe the internal charactaristics of our data itself and its external charactaristicts
4. the lineage and logs pillar describe internal dependencies withing our data and its inteactions with the external worlds
5. without any one of these four pillars we left with an incomplete picture of our data at a given point in time

- validation metrics:
  computable at training time, but more costly to optimize than training metrics


data quality checks :
	- buisness metrics
		- valid values enum or ranges
	- logic tests
		- table contaian columns x, y of type int,float

anomaly detection

- guardrail metrics
guardrail metrics can be accounted for during the model tunning process
doing so will help priduce better models which are more natully ablle to advance from development to production
considering more than one metric changes our task from finding the answer to learning what is possible

- production metrics
the metrics of most intrest to us are often unavailable during model tunning like ctr profit etc.
making final decision about deployment may invovle multiple stakeholders and expers in a project





data:
	- data drift
	- data bias

features:
	- feature enrichment
	- data pipelines issues

model
	- model performance decay
	- model bias

buisness decision
	- kpi impacts		  
	- bias related impacts
    
 https://www.montecarlodata.com/blog-the-non-engineers-guide-to-bad-data/